In [ ]:
!pip install transformers datasets pandas

In [ ]:
!noclip

/bin/bash: line 1: noclip: command not found


In [ ]:
from transformers import DebertaTokenizer, DebertaForSequenceClassification


In [ ]:
# Initialize DeBERTa tokenizer
tokenizer = DebertaTokenizer.from_pretrained('microsoft/deberta-base')

# Load the pre-trained DeBERTa model for sequence classification
model = DebertaForSequenceClassification.from_pretrained('microsoft/deberta-base', num_labels=7)  # 7 classes for bug severity


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import pandas as pd
import torch

In [ ]:
df = pd.read_excel('DL_train_test_data.xlsx')

In [ ]:
#Encode Severity Levels
df['severity_label'] = df['severity_code']

# Ensure the labels range from 0 to 6
print(df['severity_label'].unique())  # Should print [0, 1, 2, 3, 4, 5, 6]


labels = torch.tensor(df['severity_label'].values)

[2 6 1 4 5 0]


In [ ]:
# Combine the short and long descriptions into one text field
df['text'] = df['short_description'].fillna('') + ' ' + df['long_description'].fillna('')

# Tokenize the data with DeBERTa tokenizer
tokenized_data = tokenizer(
    df['text'].tolist(),
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='pt'
)


In [ ]:
from torch.utils.data import Dataset, DataLoader

class BugDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Create encodings and labels
encodings = tokenizer(df['text'].tolist(), truncation=True, padding=True, max_length=128)
labels = torch.tensor(df['severity_label'].values)

# Create Dataset and DataLoader
dataset = BugDataset(encodings, labels)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)


In [ ]:
from transformers import AdamW
from torch.nn import CrossEntropyLoss
from tqdm.notebook import tqdm

# Set up device and move model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Set up optimizer and loss function
optimizer = AdamW(model.parameters(), lr=5e-5)
loss_fn = CrossEntropyLoss()

# Training loop
epochs = 3
for epoch in range(epochs):
    model.train()
    train_loss = 0
    for batch in tqdm(train_loader):
        optimizer.zero_grad()

        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        train_loss += loss.item()

        loss.backward()
        optimizer.step()

    avg_train_loss = train_loss / len(train_loader)
    print(f"Epoch {epoch+1} | Train Loss: {avg_train_loss}")


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/2568 [00:00<?, ?it/s]

<ipython-input-25-6e98c404277e>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(self.labels[idx])


Epoch 1 | Train Loss: 0.6512069333594211


  0%|          | 0/2568 [00:00<?, ?it/s]

Epoch 2 | Train Loss: 0.6579464485294351


  0%|          | 0/2568 [00:00<?, ?it/s]

Epoch 3 | Train Loss: 0.6546906774161482


In [ ]:
model.eval()
val_loss = 0
correct_predictions = 0

with torch.no_grad():
    for batch in val_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        val_loss += outputs.loss.item()

        _, preds = torch.max(outputs.logits, dim=1)
        correct_predictions += torch.sum(preds == labels)

avg_val_loss = val_loss / len(val_loader)
accuracy = correct_predictions.double() / len(val_dataset)

print(f"Validation Loss: {avg_val_loss}")
print(f"Accuracy: {accuracy}")


<ipython-input-25-6e98c404277e>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(self.labels[idx])


Validation Loss: 0.6661033880812728
Accuracy: 0.8374878286270692


In [ ]:
torch.save(model.state_dict(), f'deberta_epoch_{epoch+1}.pth')
